In [66]:
import scanpy as sc
import pandas as pd
import seaborn as sns
import anndata as ad
import numpy as np
import collections

In [67]:
# read in data
with open("../../../Data/ROSMAP_blood_gene_expression_monocyte_ACTL_04172023.csv") as your_data:
    adata_rosmap = ad.read_csv(your_data, delimiter=',')
adata_rosmap = adata_rosmap.T
    
with open("../../../Data/ADNI_blood_gene_expression_gene_aligned_04172023.csv") as your_data:
    adata_adni = ad.read_csv(your_data, delimiter=',')
adata_adni = adata_adni.T
    
# read in label
Blood_Info = pd.read_csv("../../../Data/ROSMAP_blood_clinical_monocyte_ACTL_04172023.csv",header=0, index_col=0)

# read in label
Blood_Adni_Info = pd.read_csv("../../../Data/ADNI_NewModel_NewLabel_Clinical_Info.csv",header=0, index_col=0)

# read in projection from Monocle3
ADNI_Transferred_Label = pd.read_csv("../../../Data/Monocle_LabelTransfer_ROSMAP_ADNI_Braak.csv")

In [68]:
SID = list(ADNI_Transferred_Label['X'])
PreLabel = list(ADNI_Transferred_Label['braak_fix'])
Mapping_ADNIID_PreLabel = dict(zip(SID, PreLabel))

In [69]:
#Mapping_ADNIID_PreLabel

In [70]:
ADNI_Transferred_Label

,Unnamed: 0,X,Label,Gender,Age,Size_Factor,data_set,braak_xfr,braak_fix
0,1,116_S_1249,Asym AD,0,75,1.008569,query,5.0,5
1,2,037_S_4410,Asym AD,0,69,1.013750,query,5.0,5
2,3,006_S_4153,Asym AD,1,79,0.999273,query,5.0,5
3,4,116_S_1232,Typical AD,0,76,0.994592,query,5.0,5
4,5,099_S_4205,Asym AD,0,81,1.006435,query,NaN,5
...,...,...,...,...,...,...,...,...,...
739,740,009_S_2381,Typical AD,1,69,0.990172,query,NaN,5
740,741,053_S_4557,Typical AD,0,83,1.002712,query,5.0,5
741,742,073_S_4300,Asym AD,0,81,1.003670,query,5.0,5
742,743,041_S_4014,Asym AD,1,81,0.993808,query,5.0,5


In [71]:
Blood_Adni_Info

,Label,Gender,Age,DX,Progressed,MMSE_Month,MMSE_Close,MMSE_BL,CDRSB_BL,PTAU_Close,TAU_Close,AV45_Close,CDRSB_Month,first_changing_time,n_counts_all,braaksc_trans,NewLabel
116_S_1249,Asym AD,Female,75,CN,Yes,30.0,30.0,30.0,0.0,inf,inf,inf,0.0,35.836066,46323.804849,5.0,Others
037_S_4410,Asym AD,Female,69,CN,No,29.0,29.0,29.0,0.0,29.14,313.5,0.9862,0.0,61.475410,46533.937284,4.0,Others
006_S_4153,Asym AD,Male,79,Dementia,No,22.0,22.0,22.0,3.0,42.01,405.9,1.2779,3.0,30.000000,45881.501968,4.0,Asym AD
116_S_1232,Typical AD,Female,76,CN,No,27.0,27.0,28.0,0.0,inf,inf,1.2821,0.0,36.295082,45672.591813,4.0,Others
099_S_4205,Asym AD,Female,81,MCI,No,29.0,29.0,29.0,0.5,27.46,281.8,1.3392,0.5,61.016393,46196.449643,5.0,Asym AD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
009_S_2381,Typical AD,Male,69,MCI,Yes,28.0,28.0,28.0,2.5,57.70,576.9,1.3430,2.5,24.360656,45503.398576,1.0,Others
053_S_4557,Typical AD,Female,83,MCI,No,27.0,27.0,27.0,2.5,26.26,285.2,1.3257,2.5,59.868852,46025.165544,5.0,Asym AD
073_S_4300,Asym AD,Female,81,MCI,No,26.0,26.0,26.0,1.5,14.10,166.1,0.9873,1.5,35.803279,46121.202942,4.0,Asym AD
041_S_4014,Asym AD,Male,81,CN,No,29.0,29.0,29.0,0.0,30.92,313.2,1.6553,0.0,47.344262,45695.602901,1.0,Control


In [72]:
SID = list(ADNI_Transferred_Label.X)
Braak_trans = list(ADNI_Transferred_Label.braak_fix)

Mapping_SID_Braak = dict(zip(SID, Braak_trans))

SID = list(Blood_Adni_Info.index)
DX = list(Blood_Adni_Info.DX)
CDRSB = list(Blood_Adni_Info.CDRSB_Month)

Mapping_SID_DX = dict(zip(SID, DX))
Mapping_SID_CDRSB = dict(zip(SID, CDRSB))


In [73]:
SID = list(adata_adni.obs.index)

DXlist = list()
Braaklist = list()
CDRSBlist = list()
for st in SID:
    DXlist.append(Mapping_SID_DX[st])
    Braaklist.append(Mapping_SID_Braak[st])
    CDRSBlist.append(Mapping_SID_CDRSB[st])

In [74]:
adata_adni.obs['DX'] = DXlist
adata_adni.obs['braaksc_trans'] = Braaklist
adata_adni.obs['CDRSB_Month'] = CDRSBlist

In [75]:
adata_adni.obs

,DX,braaksc_trans,CDRSB_Month
116_S_1249,CN,5,0.0
037_S_4410,CN,5,0.0
006_S_4153,Dementia,5,3.0
116_S_1232,CN,5,0.0
099_S_4205,MCI,5,0.5
...,...,...,...
009_S_2381,MCI,5,2.5
053_S_4557,MCI,5,2.5
073_S_4300,MCI,5,1.5
041_S_4014,CN,5,0.0


# Classify

In [76]:
anndata_Target = adata_adni

In [77]:
anndata_Target.obs

,DX,braaksc_trans,CDRSB_Month
116_S_1249,CN,5,0.0
037_S_4410,CN,5,0.0
006_S_4153,Dementia,5,3.0
116_S_1232,CN,5,0.0
099_S_4205,MCI,5,0.5
...,...,...,...
009_S_2381,MCI,5,2.5
053_S_4557,MCI,5,2.5
073_S_4300,MCI,5,1.5
041_S_4014,CN,5,0.0


In [78]:
Label = list()
SID = list(anndata_Target.obs.index)

for gt in SID:
    CRD = anndata_Target.obs.loc[gt,:].CDRSB_Month
    Braak = anndata_Target.obs.loc[gt,:].braaksc_trans
    Dx = anndata_Target.obs.loc[gt,:].DX
    if CRD > 4 and Braak >=4:
        Label.append("Typical AD")
    elif CRD <= 4 and Braak >=4 and CRD > 0:
        Label.append("Asym AD")
    elif CRD > 4 and Braak <=2:
        Label.append("Low-NFT AD")
    elif CRD <= 0.5 and Braak <=2:
        Label.append("Control")
    else:
        Label.append("Others")

In [79]:
anndata_Target.obs['NewLabel'] = Label

In [80]:
df = anndata_Target.obs[anndata_Target.obs.NewLabel=='Asym AD']
counter = collections.Counter(df['DX'])
print(counter)
Acc_Asym = (counter['CN'] + counter['MCI']) / df.shape[0]
print("Asym AD transfer accuracy is ", Acc_Asym)

Counter({'MCI': 351, 'Dementia': 32, 'CN': 19})
Asym AD transfer accuracy is  0.9203980099502488


In [81]:
df = anndata_Target.obs[anndata_Target.obs.NewLabel=='Typical AD']
counter = collections.Counter(df['DX'])
print(counter)
Acc_TAD = (counter['Dementia'] + counter['MCI']) / df.shape[0]
print("Typical AD transfer accuracy is ", Acc_TAD)

Counter({'Dementia': 84, 'MCI': 28, 'CN': 17})
Typical AD transfer accuracy is  0.8682170542635659


In [82]:
df = anndata_Target.obs[anndata_Target.obs.NewLabel=='Low-NFT AD']
counter = collections.Counter(df['DX'])
print(counter)
Acc_LTau = (counter['Dementia'] + counter['MCI']) / df.shape[0]
print("Low-NFT AD transfer accuracy is ", Acc_LTau)

Counter()


ZeroDivisionError: division by zero

In [83]:
df = anndata_Target.obs[anndata_Target.obs.NewLabel=='Control']
counter = collections.Counter(df['DX'])
print(counter)
Acc_CN = (counter['CN']) / df.shape[0]
print("Control transfer accuracy is ", Acc_CN)

Counter()


ZeroDivisionError: division by zero

In [ ]:
df = anndata_Target.obs[anndata_Target.obs.NewLabel=='Others']
counter = collections.Counter(df['DX'])
print(counter)

In [ ]:
Acc_Avg = (Acc_Asym + Acc_CN + Acc_LTau + Acc_TAD)/4
print("Avg acc is ", Acc_Avg)

In [ ]:
counter = collections.Counter(Label)
print(counter)

In [ ]:
len(Label)-247

In [6]:
import collections
# For rosmap blood
Blood_ROSMAP_Info = Blood_Info
BloodSampleID_Rosmap = list(adata_rosmap.obs.index)

SID = list(Blood_ROSMAP_Info.specimenID)
ADType = list(Blood_ROSMAP_Info.Group1)
Mapping = {'Control': 0, 'Typical AD': 1, 'Asym AD': 2, 'Low-NFT AD': 3}
frequency_rosmap = collections.Counter(ADType)
print(dict(frequency_rosmap))

Rosmap_Sample_CN = list()
Rosmap_Sample_TAD = list()
Rosmap_Sample_AAD = list()
for i in range(len(SID)):
    if ADType[i] == 'Control':
        Rosmap_Sample_CN.append(SID[i])
    if ADType[i] == 'Typical AD' or ADType[i] == 'Low-NFT AD':
        Rosmap_Sample_TAD.append(SID[i])
    if ADType[i] == 'Asym AD':
        Rosmap_Sample_AAD.append(SID[i])
#print(Rosmap_Sample_CN)
#print(Rosmap_Sample_TAD)
#print(Rosmap_Sample_AAD)

SType_Blood_Rosmap = list()
for i in range(len(BloodSampleID_Rosmap)):
    if BloodSampleID_Rosmap[i] in SID:
        indt = SID.index(BloodSampleID_Rosmap[i])
        tmp = Mapping[ADType[indt]]
        SType_Blood_Rosmap.append(tmp)
print(SType_Blood_Rosmap)
print(len(SType_Blood_Rosmap))

{'Typical AD': 46, 'Asym AD': 88, 'Control': 18, 'Low-NFT AD': 5}
[1, 2, 2, 0, 1, 2, 1, 0, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 0, 1, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 1, 2, 2, 2, 1, 0, 0, 2, 2, 2, 1, 1, 2, 2, 0, 2, 3, 2, 1, 0, 2, 2, 3, 1, 0, 0, 2, 2, 2, 2, 3, 0, 0, 2, 2, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 0, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 0, 0, 1, 2, 2, 2, 2, 3, 1, 2, 2, 2, 3, 2, 2]
157


In [7]:
Adni_SID = list(Blood_Adni_Info.PTID)
Adni_Type = list(Blood_Adni_Info.DX)

SID_data = list(adata_adni.obs.index)

Adni_Type_Data = list()
for i in range(len(SID_data)):
    if SID_data[i] in Adni_SID:
        indt = Adni_SID.index(SID_data[i])
        tmp = Adni_Type[indt]
        Adni_Type_Data.append(tmp)
print(Adni_Type_Data)

['CN', 'CN', 'Dementia', 'CN', 'MCI', 'MCI', 'CN', 'MCI', 'CN', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'CN', 'CN', 'MCI', 'MCI', 'CN', 'CN', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'Dementia', 'Dementia', 'MCI', 'MCI', 'MCI', 'Dementia', 'MCI', 'MCI', 'Dementia', 'MCI', 'MCI', 'Dementia', 'MCI', 'MCI', 'CN', 'CN', 'CN', 'MCI', 'CN', 'CN', 'MCI', 'CN', 'Dementia', 'CN', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'CN', 'CN', 'CN', 'MCI', 'CN', 'CN', 'MCI', 'MCI', 'CN', 'MCI', 'MCI', 'CN', 'Dementia', 'CN', 'CN', 'MCI', 'MCI', 'CN', 'CN', 'CN', 'MCI', 'MCI', 'Dementia', 'CN', 'MCI', 'Dementia', 'CN', 'CN', 'CN', 'MCI', 'MCI', 'MCI', 'CN', 'CN', 'MCI', 'CN', 'MCI', 'CN', 'Dementia', 'MCI', 'MCI', 'MCI', 'MCI', 'CN', 'Dementia', 'MCI', 'MCI', 'MCI', 'MCI', 'Dementia', 'Dementia', 'MCI', 'CN', 'Dementia', 'MCI', 'MCI', 'CN', 'CN', 'CN', 'MCI', 'MCI', 'MCI', 'CN', 'MCI', 'MCI', 'MCI', 'Dementia', 'MCI', 'CN', 'MCI', 'MCI', 'MCI', 'CN', 'CN', 'CN', 'MCI', 'Dementia', 'CN', 'MCI', 'MCI', 'CN', 'MC

In [8]:
# convert predited label
Mapping = {'Control': 0, 'Typical AD': 1, 'Asym AD': 2, 'Low-NFT AD': 3}
y_pred = np.zeros(len(SID_data))
for i in range(len(SID_data)):
    y_pred[i] = Mapping[Mapping_ADNIID_PreLabel[SID_data[i]]]
    

In [9]:
Mapping = {'CN': 0, 'Dementia': 1, 'MCI': 2}
y_true_lb = Adni_Type_Data
y_true = np.zeros(len(y_true_lb))
for i in range(len(y_true_lb)):
    y_true[i] = Mapping[y_true_lb[i]]

In [10]:
pred_ind_cn = np.where(y_pred==0)[0]
true_ind_cn = np.where(y_true==0)[0]
overlap = list(set(pred_ind_cn) & set(true_ind_cn))
if len(pred_ind_cn) == 0:
    print("precision of CN is 0")
else:
    print("precision of CN is ", len(overlap)/len(pred_ind_cn))

pred_ind_tad = np.where(y_pred==1)[0]
true_ind_tad = np.where(y_true==1)[0]
overlap = list(set(pred_ind_tad) & set(true_ind_tad))
print("precision of TAD is ", len(overlap)/len(pred_ind_tad))

pred_ind_aad = np.where(y_pred==2)[0]
true_ind_aad = np.where(y_true==2)[0]
overlap = list(set(pred_ind_aad) & set(true_ind_aad))
print("precision of AAD is ", len(overlap)/len(pred_ind_aad))

precision of CN is 0
precision of TAD is  0.15660919540229884
precision of AAD is  0.5208333333333334


In [11]:
pred_ind_cn = np.where(y_pred==0)[0]
true_ind_cn = np.where(y_true==0)[0]
overlap = list(set(pred_ind_cn) & set(true_ind_cn))
#if len(pred_ind_cn) == 0:
#    print("precision of CN is 0")
#else:
#    print("precision of CN is ", len(overlap)/len(pred_ind_cn))


pred_ind_aad = np.where(y_pred==2)[0]
true_ind_aad = np.where(y_true==2)[0]
overlap = list(set(pred_ind_aad) & set(true_ind_aad))
#print("precision of AAD is ", len(overlap)/len(pred_ind_aad))

pred_ind_tad = np.where(y_pred==1)[0]
true_ind_tad = np.where(y_true==1)[0]
true_ind_tad_aad = list(true_ind_aad) + list(true_ind_tad)
overlap = list(set(pred_ind_tad) & set(true_ind_tad_aad))
print("precision of TAD is ", len(overlap)/len(pred_ind_tad))

pred_ind_aad_cn = list(pred_ind_aad) #+ list(pred_ind_cn)
true_ind_aad_cn = list(true_ind_cn) + list(true_ind_aad)
overlap = list(set(pred_ind_aad_cn) & set(true_ind_aad_cn))
print("precision of AAD  is ", len(overlap)/len(pred_ind_aad_cn))

pred_ind_aad_cn = list(pred_ind_cn) #list(pred_ind_aad) + 
true_ind_aad_cn = list(true_ind_cn) + list(true_ind_aad)
overlap = list(set(pred_ind_aad_cn) & set(true_ind_aad_cn))
if len(pred_ind_aad_cn) == 0:
    print("precision of CN  is 0")
else:
    print("precision of CN  is ", len(overlap)/len(pred_ind_aad_cn))

pred_ind_lt = np.where(y_pred==3)[0]
true_ind_lt = np.where(y_true==1)[0]
true_ind_lt_aad = list(true_ind_aad) + list(true_ind_lt)
overlap = list(set(pred_ind_lt) & set(true_ind_lt_aad))
if len(pred_ind_lt) == 0:
    print("precision of LowTau is 0")
else:
    print("precision of LowTau is ", len(overlap)/len(pred_ind_lt))

precision of TAD is  0.6695402298850575
precision of AAD  is  0.8541666666666666
precision of CN  is 0
precision of LowTau is 0
